In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import numpy as np
data_dir = 'gdrive/My Drive/EEG_project/'
X_test = np.load(data_dir + "X_test.npy")
y_test = np.load(data_dir + "y_test.npy")
person_train_valid = np.load(data_dir + "person_train_valid.npy")
X_train_valid = np.load(data_dir + "X_train_valid.npy")
y_train_valid = np.load(data_dir + "y_train_valid.npy")
person_test = np.load(data_dir +"person_test.npy")

Mounted at /content/gdrive


In [2]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [0]:
y_train_valid -= 769
y_test -= 769
person_train_valid = person_train_valid.astype(int)


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# datatype = torch.float
# torch.cuda.device(device)
# print(device)

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Temporal Convolution
        self.conv1 = nn.Conv2d(1, 40, (1, 40), stride = (1, 2))
        nn.init.xavier_uniform_(self.conv1.weight)
        # Batch Normalization
        self.batch1 = nn.BatchNorm2d(40)
        # ELU
        self.elu1 = nn.ELU()
        # Spatial Convolution
        self.conv2 = nn.Conv2d(40, 40, (22, 1), stride = (1, 1))
        nn.init.xavier_uniform_(self.conv2.weight)
        # Batch Normalization
        self.batch2 = nn.BatchNorm2d(40)
        # ELU
        self.elu2 = nn.ELU()
        # Avg Pooling
        self.pool1 = nn.AvgPool2d((1, 75), stride = (1, 15))
        # Dropout
        self.dropout = nn.Dropout(p = 0.3)
        # Fully Connected
        self.fc1 = nn.Linear(40 * 28, 4)
        # Softmax
        self.softmax = nn.Softmax(1)
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.elu1(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = self.elu2(x)
        x = self.pool1(x)
        x = self.dropout(x)
        x = x.reshape(x.shape[0], x.shape[1] * x.shape[2] * x.shape[3])
        x = self.fc1(x)
        x = self.softmax(x)
        return x

In [0]:
cnn = CNN()

In [0]:
lr = 0.005
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=lr)

In [0]:
def X_reshape(X, index, y):#y is person_train_valid
    arr = np.array([])
    for i in range(len(X)):
        if(y[i][0] == index):
            if(len(arr) == 0):
                arr = np.expand_dims(X[i], axis=0)
            else:
                arr = np.concatenate((arr, np.expand_dims(X[i], axis=0)), axis=0)
    return arr


In [0]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 4)

In [11]:
for ind in range(9):
    X_train_valid_reshape = X_reshape(X_train_valid, ind, person_train_valid)
    y_train_valid_reshape = X_reshape(y_train_valid, ind, person_train_valid)
    batch_size = 500
    num_iters = 1
    num_epoch = 30
    epoch_loss = [0.0 for i in range(num_epoch)]
    for epoch in range(num_epoch):
        counter = 0
        for train_index, valid_index in kf.split(X_train_valid_reshape):
            X_train_cur = torch.tensor(X_train_valid_reshape[train_index].reshape(X_train_valid_reshape[train_index].shape[0], -1, 22, 1000), dtype=torch.float)
            X_valid_cur = torch.tensor(X_train_valid_reshape[valid_index].reshape(X_train_valid_reshape[valid_index].shape[0], -1, 22, 1000), dtype=torch.float)
            y_train_cur = torch.tensor(y_train_valid_reshape[train_index], dtype=torch.long)
            y_valid_cur = torch.tensor(y_train_valid_reshape[valid_index], dtype=torch.long)
            for it in np.arange(num_iters):
                index = np.random.choice(X_train_cur.shape[0], batch_size)
                X_batch = X_train_cur[index, :, :, :]
                y_batch = y_train_cur[index]
                optimizer.zero_grad()
                outputs = cnn(X_batch)

                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                epoch_loss[epoch] += loss

                counter += 1
            # Validation Accuracy for current K-fold
            output_val = cnn(X_valid_cur)
            prediction_val = torch.argmax(output_val, axis = 1)
            accuracy_val = float(prediction_val.eq(y_valid_cur).sum()) / float(y_valid_cur.shape[0])
            print('Current Epoch: {}, Validation Accuracy: {}'.format(epoch, accuracy_val))
        epoch_loss[epoch] /= counter 
        print('Current Subject: {}, Current Epoch: {}, Epoch Loss: {}'.format(ind, epoch, epoch_loss[epoch]))
    X_test_reshape = X_reshape(X_test, ind, person_test)
    y_test_reshape = X_reshape(y_test, ind, person_test)
    X_test_reshape = torch.tensor(X_test_reshape.reshape(X_test_reshape.shape[0], -1, 22, 1000), dtype= torch.float)
    y_test_reshape = torch.tensor(y_test_reshape, dtype = torch.long)
    output_test = cnn(X_test_reshape)
    prediction_test = torch.argmax(output_test, axis = 1)
    accuracy_test = float(prediction_test.eq(y_test_reshape).sum()) / float(y_test_reshape.shape[0])
    print('Current Subject: {}, Current Epoch: {}, Test Accuracy: {}'.format(ind, epoch, accuracy_test))
    model_name = 'model'+str(ind)+'.pt'
    torch.save(cnn, model_name)


Current Epoch: 0, Validation Accuracy: 0.16666666666666666
Current Epoch: 0, Validation Accuracy: 0.3220338983050847
Current Epoch: 0, Validation Accuracy: 0.3728813559322034
Current Epoch: 0, Validation Accuracy: 0.5084745762711864
Current Subject: 0, Current Epoch: 0, Epoch Loss: 1.35331130027771
Current Epoch: 1, Validation Accuracy: 0.4666666666666667
Current Epoch: 1, Validation Accuracy: 0.576271186440678
Current Epoch: 1, Validation Accuracy: 0.5084745762711864
Current Epoch: 1, Validation Accuracy: 0.6101694915254238
Current Subject: 0, Current Epoch: 1, Epoch Loss: 1.236122488975525
Current Epoch: 2, Validation Accuracy: 0.5166666666666667
Current Epoch: 2, Validation Accuracy: 0.7457627118644068
Current Epoch: 2, Validation Accuracy: 0.5932203389830508
Current Epoch: 2, Validation Accuracy: 0.6779661016949152
Current Subject: 0, Current Epoch: 2, Epoch Loss: 1.152318000793457
Current Epoch: 3, Validation Accuracy: 0.65
Current Epoch: 3, Validation Accuracy: 0.7966101694915254

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Current Epoch: 0, Validation Accuracy: 0.3389830508474576
Current Epoch: 0, Validation Accuracy: 0.3898305084745763
Current Epoch: 0, Validation Accuracy: 0.559322033898305
Current Epoch: 0, Validation Accuracy: 0.3389830508474576
Current Subject: 1, Current Epoch: 0, Epoch Loss: 1.3208836317062378
Current Epoch: 1, Validation Accuracy: 0.4576271186440678
Current Epoch: 1, Validation Accuracy: 0.4576271186440678
Current Epoch: 1, Validation Accuracy: 0.576271186440678
Current Epoch: 1, Validation Accuracy: 0.559322033898305
Current Subject: 1, Current Epoch: 1, Epoch Loss: 1.256429672241211
Current Epoch: 2, Validation Accuracy: 0.4745762711864407
Current Epoch: 2, Validation Accuracy: 0.5084745762711864
Current Epoch: 2, Validation Accuracy: 0.5423728813559322
Current Epoch: 2, Validation Accuracy: 0.559322033898305
Current Subject: 1, Current Epoch: 2, Epoch Loss: 1.208672285079956
Current Epoch: 3, Validation Accuracy: 0.576271186440678
Current Epoch: 3, Validation Accuracy: 0.57627